In [1]:
from policy import PIB, NEATProperty, NEATPolicy, NEATRequest
from cib import CIB

# Application Request

We consider an application that would like to open a new TCP connection using NEAT to a destination host `d1` with the IP 10.1.23.45. Further the MTU should be larger than 1500 bytes if possible. 

We define three properties to represent these application requirements and combine them into a NEATRequest: 

In [2]:
property1 = NEATProperty(('remote_ip', '10.1.23.45'), precedence=NEATProperty.IMMUTABLE)
property2 = NEATProperty(('MTU', (1500, float('inf'))), precedence=NEATProperty.REQUESTED) 
property3 = NEATProperty(('transport_TCP', True))  # REQUESTED is the default property precedence

request = NEATRequest()
request.properties.insert(property1)
request.properties.insert(property2)
request.properties.insert(property3)

print(request)
request.properties

<NEATRequest: 0 candidates, 3 properties>


{(transport_TCP|True), [remote_ip|10.1.23.45], (MTU|1500-inf)}

For this scenario we map the requirement `MTU>1500` to the range `[1500, inf]`. 

Properties can be exported to JSON strings (and imported):

In [3]:
request.properties.json()

'[{"transport_TCP": {"precedence": 1, "score": NaN, "value": true}}, {"remote_ip": {"precedence": 2, "score": NaN, "value": "10.1.23.45"}}, {"MTU": {"precedence": 1, "score": NaN, "value": [1500, Infinity]}}]'

# Exemplary Setup

Consider a host with three local interfaces `en0`, `en1`, `ra0`. Two of the interfaces, `en0`, `en1`, are wired while `ra0` is a 3G interface. We populate an instance of the Characteristic Information Base (CIB) with some information about the host interfaces and the network.

In [4]:
cib = CIB('cib/example/')

loading CIB source A.connection
loading CIB source B.connection
loading CIB source C.connection


The currently known network chracteristics are stored as entries in the CIB, where each entry contains a set of properties associated with some interface:

In [5]:
cib.dump()

A: {[local_ip|10.2.0.1], [remote_ip|10.1.23.45], [capacity|10000], [interface|en0], [is_wired|True], [MTU|9600], <transport|TCP>, <dns_name|backup.example.com>}
B: {[local_ip|192.168.1.2], [capacity|40000], [interface|en1], [is_wired|True], [MTU|1500], <transport_TCP|True>, <transport_UDP|True>}
C: {[local_ip|10.10.2.2], [remote_ip|10.1.23.45], [capacity|60], [interface|ra0], [is_wired|False], [MTU|890]}


# PIB 
We define two sample policies and add them to the Policy Information Base (PIB).

A "bulk transfer" policy is configured which is triggered by a specific destination IP, which is known to be the address of backup NFS share:

In [6]:
policy1 = NEATPolicy(name='Bulk transfer')
policy1.match.insert(NEATProperty(('remote_ip', '10.1.23.45')))
policy1.properties.insert(NEATProperty(('capacity', (10000, 100000)), precedence=NEATProperty.IMMUTABLE))
policy1.properties.insert(NEATProperty(('MTU', 9600)))

Another policy is in place to enable TCP window scaling on 10G links (if possible):

In [7]:
policy2 = NEATPolicy(name='TCP options')
policy2.match.insert(NEATProperty(('MTU', 9600)))
policy2.match.insert(NEATProperty(('is_wired', True)))
policy2.properties.insert(NEATProperty(('TCP_window_scale', True)))

In [8]:
pib = PIB()
pib.register(policy1)
pib.register(policy2)
pib.dump()

===== PIB START =====
POLICY Bulk transfer: {(remote_ip|10.1.23.45)}  ==>  {(MTU|9600), [capacity|10000-100000]}
POLICY TCP options: {(is_wired|True), (MTU|9600)}  ==>  {(TCP_window_scale|True)}
===== PIB END =====


# Lookup Result

## CIB Lookup

First a lookup in the CIB is performed. Our NEAT request yields three candidates:


In [9]:
cib.lookup(request)
request.dump()

{(transport_TCP|True), [remote_ip|10.1.23.45], (MTU|1500-inf)}
===== candidates =====
[0]PROPERTIES: {[local_ip|10.10.2.2], [remote_ip|10.1.23.45]+1.0, [capacity|60], [interface|ra0], [is_wired|False], [MTU|890]-1.0, (transport_TCP|True)}, POLICIES: set()
[1]PROPERTIES: {[local_ip|10.2.0.1], [remote_ip|10.1.23.45]+1.0, [capacity|10000], [interface|en0], [is_wired|True], (transport_TCP|True), [MTU|9600]+1.0, <transport|TCP>, <dns_name|backup.example.com>}, POLICIES: set()
[2]PROPERTIES: {[local_ip|192.168.1.2], [remote_ip|10.1.23.45], [capacity|40000], [interface|en1], [is_wired|True], [MTU|1500]+1.0, (transport_TCP|True)+1.0, <transport_UDP|True>}, POLICIES: set()
===== candidates =====


Each candidate is comprised of the union of the properties of a single CIB entry and the application request. Whenever  the two sets intersect, the values of the corresponding properties are compared. If two properties match, the associated candidate property score is increased (e.g., `[MTU|1500]+1.0` indicates a new score of 1.0). The score is decreased if there is a mismatch in the property values.

## PIB Lookup
In the next step the policies are applied starting with the "Bulk transfer" policy which has the *smallest* number of match entries.


In [10]:
pib.lookup_all(request.candidates)
request.dump()

Candidate 0 is invalidated due to policy
{(transport_TCP|True), [remote_ip|10.1.23.45], (MTU|1500-inf)}
===== candidates =====
[0]PROPERTIES: {[local_ip|10.2.0.1], [remote_ip|10.1.23.45]+1.0, [capacity|10000]+1.0, [interface|en0], (TCP_window_scale|True), [is_wired|True], (transport_TCP|True), [MTU|9600]+2.0, <transport|TCP>, <dns_name|backup.example.com>}, POLICIES: {4344536648, 4344536816}
[1]PROPERTIES: {[local_ip|192.168.1.2], [remote_ip|10.1.23.45], [capacity|40000]+1.0, [interface|en1], (TCP_window_scale|True), [is_wired|True], [MTU|1500]+0.0, (transport_TCP|True)+1.0, <transport_UDP|True>}, POLICIES: {4344536648, 4344536816}
===== candidates =====


Candidate 1 becomes:        

In [11]:
request.candidates[0].dump()

PROPERTIES: {[local_ip|10.2.0.1], [remote_ip|10.1.23.45]+1.0, [capacity|10000]+1.0, [interface|en0], (TCP_window_scale|True), [is_wired|True], (transport_TCP|True), [MTU|9600]+2.0, <transport|TCP>, <dns_name|backup.example.com>}, POLICIES: {4344536648, 4344536816}


Next we examine Candidate 2:

In [12]:
request.candidates[1].dump()

PROPERTIES: {[local_ip|192.168.1.2], [remote_ip|10.1.23.45], [capacity|40000]+1.0, [interface|en1], (TCP_window_scale|True), [is_wired|True], [MTU|1500]+0.0, (transport_TCP|True)+1.0, <transport_UDP|True>}, POLICIES: {4344536648, 4344536816}


Note that the score of the MTU property was reduced, as it did not match the requested property of the "Bulk transfer" policy.

The "TCP options" policy is not applied as the candidate does not match the policy's MTU property.

---

The third candidate was invalidated because the "Bulk transfer" policy contains an immutable property requiring a capacity of 10G, which candidate 3 cannot fulfil.

---

# NEAT Logic
The two candidates can now be passed on to the NEAT logic as JSON strings

In [13]:
request.candidates[0].properties.json()

'[{"local_ip": {"precedence": 2, "score": NaN, "value": "10.2.0.1"}}, {"remote_ip": {"precedence": 2, "score": 1.0, "value": "10.1.23.45"}}, {"capacity": {"precedence": 2, "score": 1.0, "value": 10000}}, {"interface": {"precedence": 2, "score": NaN, "value": "en0"}}, {"TCP_window_scale": {"precedence": 1, "score": NaN, "value": true}}, {"is_wired": {"precedence": 2, "score": NaN, "value": true}}, {"transport_TCP": {"precedence": 1, "score": NaN, "value": true}}, {"MTU": {"precedence": 2, "score": 2.0, "value": 9600}}, {"transport": {"precedence": 0, "score": NaN, "value": "TCP"}}, {"dns_name": {"precedence": 0, "score": NaN, "value": "backup.example.com"}}]'

In [14]:
request.candidates[1].properties.json()

'[{"local_ip": {"precedence": 2, "score": NaN, "value": "192.168.1.2"}}, {"remote_ip": {"precedence": 2, "score": NaN, "value": "10.1.23.45"}}, {"capacity": {"precedence": 2, "score": 1.0, "value": 40000}}, {"interface": {"precedence": 2, "score": NaN, "value": "en1"}}, {"TCP_window_scale": {"precedence": 1, "score": NaN, "value": true}}, {"is_wired": {"precedence": 2, "score": NaN, "value": true}}, {"MTU": {"precedence": 2, "score": 0.0, "value": 1500}}, {"transport_TCP": {"precedence": 1, "score": 1.0, "value": true}}, {"transport_UDP": {"precedence": 0, "score": NaN, "value": true}}]'